In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import random
import json
import torch
from PIL import Image
from transformers import Blip2ForConditionalGeneration, AutoProcessor # InstructBLIP uses Blip2ForConditionalGeneration class
import pandas as pd
import accelerate
import transformers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-05-27 16:53:17.650144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748364797.838025      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748364797.901948      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/images/stacked circle feast.png
/kaggle/input/images/cheesy bake mess.png
/kaggle/input/images/green swirl delight.png
/kaggle/input/images/white puff plate.png
/kaggle/input/images/melty crunch.png
/kaggle/input/images/soupy beans.png
/kaggle/input/images/yellow spirals.png
/kaggle/input/images/goey dark bomb.png
/kaggle/input/images/bready tubes.png
/kaggle/input/images/choco fluff.png
/kaggle/input/images/stackysyrupthing.png
/kaggle/input/images/bun crunch.png
/kaggle/input/data-json/data.json
/kaggle/input/test-set/butter potato.png
/kaggle/input/test-set/stuffed rice folded.png
/kaggle/input/test-set/crunchy yellow sticks.png
/kaggle/input/test-set/sticky sweet swirl.png
/kaggle/input/test-set/spicey cruncy chinese.png


In [2]:
json_path = "/kaggle/input/data-json/data.json" # Path to your few-shot data
few_shot_image_folder = "/kaggle/input/images"  # Folder for few-shot example images
test_image_folder = "/kaggle/input/test-set"    # Folder for NEW test set images

In [3]:
model_id = "Salesforce/instructblip-flan-t5-xl"
model = Blip2ForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16, 
)
processor = AutoProcessor.from_pretrained(model_id)
print("Model loaded")

config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

You are using a model of type instructblip to instantiate a model of type blip-2. This is not supported for all configurations of models and can yield errors.


model.safetensors.index.json:   0%|          | 0.00/135k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

Model loaded


In [4]:
few_shot_data = []
try:
    with open(json_path, "r") as f:
        few_shot_data = json.load(f)
    print(f"Loaded {len(few_shot_data)} recipes from {json_path} for few-shot examples.")
except FileNotFoundError:
    print(f"Error: JSON file not found at {json_path}. Please ensure the path is correct.")
    exit()

test_data = [
    {
        "image": "butter potato.png",
        "true_title": "Butter Potato",
        "manual_summary": [
            "Bake potatoes until soft.",
            "Cut them open, fluff with a fork, and add butter and seasonings.",
            "Serve hot with desired toppings."
        ]
    },
    {
        "image": "crunchy yellow sticks.png",
        "true_title": "Crunchy Yellow Sticks",
        "manual_summary": [
            "Cut potatoes into thin sticks and soak in cold water.",
            "Deep-fry in hot oil until golden brown and crispy.",
            "Season with salt and serve immediately with ketchup."
        ]
    },
    {
        "image": "spicey cruncy chinese.png",
        "true_title": "Spicy Crunchy Chinese",
        "manual_summary": [
            "Deep-fry chicken pieces until crisp.",
            "Sauté with ginger, garlic, and chilies in a tangy, spicy sauce.",
            "Garnish with spring onions and serve hot."
        ]
    },
    {
        "image": "sticky sweet swirl.png",
        "true_title": "Sticky Sweet Swirl",
        "manual_summary": [
            "Prepare a soft dough and roll it flat.",
            "Spread with cinnamon-sugar filling and roll tightly, then slice.",
            "Bake until golden and drizzle with a sweet glaze."
        ]
    },
    {
        "image": "stuffed rice folded.png",
        "true_title": "Stuffed Rice Folded",
        "manual_summary": [
            "Prepare a savory minced meat or vegetable filling.",
            "Enclose the filling in thin dough wrappers, shaping into pockets.",
            "Steam until fully cooked and serve with a spicy dipping sauce."
        ]
    }
]
if not few_shot_data:
    print("No few-shot data")
    exit()
if not test_data:
    print("No test data ")
    exit()

print(f"Using {len(few_shot_data)} data.json few-shot examples.")
print(f"Using {len(test_data)} testing (from {test_image_folder}).")

Loaded 12 recipes from /kaggle/input/data-json/data.json for few-shot examples.
Using 12 data.json few-shot examples.
Using 5 testing (from /kaggle/input/test-set).


In [5]:
def summary_few_shot(image_path, dish_title, model, processor, few_shot_examples_data, total_few_shot_examples_in_prompt=3):
    try:
        image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        print(f"Warning: Test image not found at {image_path}. Returning 'Image not found.'")
        return "Image not found."
    selected_few_shot_examples = random.sample(few_shot_examples_data, min(total_few_shot_examples_in_prompt, len(few_shot_examples_data)))
    full_prompt = ""
    for example in selected_few_shot_examples:
        example_summary_str = "\n".join([f"{i+1}. {step}" for i, step in enumerate(example['manual_summary'])])
        full_prompt += (
            f"Question: Describe the exact cooking steps for the dish '{example['true_title']}'. "
            f"Provide precisely 3, short, imperative sentences, formatted as a numbered list. "
            f"Start directly with '1. '. Do not include any introductory phrases, concluding remarks, or conversational text. "
            f"Each step should be a single, action-oriented sentence.\n"
            f"Answer:\n{example_summary_str}\n"
        )
    full_prompt += (
        f"Question: Describe the exact cooking steps for the dish '{dish_title}' shown in the image. "
        f"Provide precisely 3, short, imperative sentences, formatted as a numbered list. "
        f"Start directly with '1. '. Do not include any introductory phrases, concluding remarks, or conversational text. "
        f"Each step should be a single, action-oriented sentence.\n"
        f"Answer:\n"
    )
    inputs = processor(images=image, text=full_prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        pixel_values=inputs.pixel_values,
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=80,
        num_beams=5,
        do_sample=False,
        early_stopping=True
    )
    generated_text = processor.decode(output[0], skip_special_tokens=True).strip()
#output was sometimes returning multiple when I varied temp and params. it is (list)
    if "Answer:" in generated_text:
        generated_text = generated_text.rsplit("Answer:", 1)[-1].strip()
    if "Question:" in generated_text:
        generated_text = generated_text.rsplit("Question:", 1)[-1].strip()
    lines = generated_text.split('\n')
    filtered_lines = [line.strip() for line in lines if line.strip() and (line.strip().startswith(tuple(str(i) + '.' for i in range(1, 4))) or len(line.strip()) > 5)]
    parsed_summary = "\n".join(filtered_lines[:3]) if filtered_lines else generated_text

    return parsed_summary

In [6]:
def summary_zero_shot(image_path, dish_title, model, processor):
    try:
        image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        print(f"Warning: Image not found at {image_path}. Returning 'Image not found.'")
        return "Image not found."
    full_prompt = (
        f"Question: Describe the exact cooking steps for the dish '{dish_title}' shown in the image. "
        f"Provide precisely 3, short, imperative sentences, formatted as a numbered list. "
        f"Start directly with '1. '. Do not include any introductory phrases, concluding remarks, or conversational text. "
        f"Each step should be a single, action-oriented sentence.\n"
        f"Answer:\n"
    )

    inputs = processor(images=image, text=full_prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        pixel_values=inputs.pixel_values,
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask ,#ye default values hai, do not change to ** input gives model_kwargs error
        max_new_tokens=100,
        num_beams=5,
        do_sample=False,
        early_stopping=True
    )
    generated_text = processor.decode(output[0], skip_special_tokens=True).strip()
    if "Answer:" in generated_text:
        generated_text = generated_text.rsplit("Answer:", 1)[-1].strip()
    if "Question:" in generated_text:
        generated_text = generated_text.rsplit("Question:", 1)[-1].strip()
    lines = generated_text.split('\n')
    # Filter for lines that start with a number (1., 2., 3.) or have some content
    filtered_lines = [line.strip() for line in lines if line.strip() and (line.strip().startswith(tuple(str(i) + '.' for i in range(1, 4))) or len(line.strip()) > 5)]
    parsed_summary = "\n".join(filtered_lines[:3]) if filtered_lines else generated_text

    return parsed_summary

In [7]:
few_shot_results = []
print("Generating cooking summaries (Few-Shot Inference on New Test Set):")
TOTAL_TEXT_FEW_SHOT_COUNT = 3 

for item in test_data:
    title = item.get("true_title", "Unknown")
    image_filename = item["image"]
    image_path = os.path.join(test_image_folder, image_filename)
    ground_truth_summary = "\n".join(item["manual_summary"])
    print(f" Processing: {title} ({image_filename})")
    instructblip_summary = summary_few_shot( 
        image_path,
        title,
        model,
        processor,
        few_shot_data, # Pass all few-shot examples for random selection
        total_few_shot_examples_in_prompt=TOTAL_TEXT_FEW_SHOT_COUNT
    )
    few_shot_results.append({
        "title": title,
        "image": image_filename,
        "ground_truth_summary": ground_truth_summary,
        "instructblip_summary": instructblip_summary 
    })

    print("Ground Truth:", ground_truth_summary)
    print("Generated Summary:" ,instructblip_summary)

Generating cooking summaries (Few-Shot Inference on New Test Set):
 Processing: Butter Potato (butter potato.png)
Ground Truth: Bake potatoes until soft.
Cut them open, fluff with a fork, and add butter and seasonings.
Serve hot with desired toppings.
Generated Summary: 1. Preheat oven to 375 degrees F. Line a baking sheet with parchment paper. 2. In a large bowl, combine butter, flour, salt, pepper, and paprika. 3. In a separate bowl, whisk together eggs, milk, buttermilk, and sour cream. 4. Pour the wet ingredients into the dry ingredients and stir with a wooden spoon
 Processing: Crunchy Yellow Sticks (crunchy yellow sticks.png)
Ground Truth: Cut potatoes into thin sticks and soak in cold water.
Deep-fry in hot oil until golden brown and crispy.
Season with salt and serve immediately with ketchup.
Generated Summary: 1. Heat oil in a large frying pan over medium heat. Add onion and garlic and cook for 2 minutes. 2. Add tomatoes and cook for 2 minutes. 3. Add spices and cook for 2 min

In [ ]:
zero_shot_results = []
print("Generating cooking summaries (Zero-Shot Inference on New Test Set):")

for item in test_data:
    title = item.get("true_title", "Unknown")
    image_filename = item["image"]
    image_path = os.path.join(test_image_folder, image_filename)
    ground_truth_summary = "\n".join(item["manual_summary"])

    print(f" Processing: {title} ({image_filename})")
    instructblip_summary = summary_zero_shot(
        image_path,
        title,
        model,
        processor
    )

    zero_shot_results.append({
        "title": title,
        "image": image_filename,
        "ground_truth_summary": ground_truth_summary,
        "instructblip_summary": instructblip_summary 
    })

    print("Ground Truth:",ground_truth_summary)
    print("Generated Summary:", instructblip_summary)

Generating cooking summaries (Zero-Shot Inference on New Test Set):
 Processing: Butter Potato (butter potato.png)
Ground Truth: Bake potatoes until soft.
Cut them open, fluff with a fork, and add butter and seasonings.
Serve hot with desired toppings.
Generated Summary: a person is preparing a dish of mashed potatoes
 Processing: Crunchy Yellow Sticks (crunchy yellow sticks.png)


In [ ]:
!pip install -U accelerate transformers nltk rouge-score
import nltk
try:
    nltk.data.find('punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')

In [ ]:
# from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
# from rouge_score import rouge_scorer

# # --- BLEU and ROUGE Score Calculation ---
# print("\n" + "="*50)
# print("Calculating BLEU and ROUGE Scores:")
# print("="*50 + "\n")

# # Initialize ROUGE scorer
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# # Prepare data for scoring
# few_shot_references = [nltk.word_tokenize(item['ground_truth_summary'].lower()) for item in few_shot_results]
# few_shot_hypotheses = [nltk.word_tokenize(item['instructblip_summary'].lower()) for item in few_shot_results]

# zero_shot_references = [nltk.word_tokenize(item['ground_truth_summary'].lower()) for item in zero_shot_results]
# zero_shot_hypotheses = [nltk.word_tokenize(item['instructblip_summary'].lower()) for item in zero_shot_results]

# # --- Calculate BLEU Scores ---
# # Using SmoothingFunction() to handle cases where there are no common n-grams (prevents zero division)
# smooth = SmoothingFunction().method1

# # Few-shot BLEU
# bleu_few_shot_scores = []
# for ref, hyp in zip(few_shot_references, few_shot_hypotheses):
#     # BLEU expects a list of reference sentences (even if only one)
#     bleu_few_shot_scores.append(sentence_bleu([ref], hyp, smoothing_function=smooth))
# avg_bleu_few_shot = sum(bleu_few_shot_scores) / len(bleu_few_shot_scores) if bleu_few_shot_scores else 0

# # Zero-shot BLEU
# bleu_zero_shot_scores = []
# for ref, hyp in zip(zero_shot_references, zero_shot_hypotheses):
#     bleu_zero_shot_scores.append(sentence_bleu([ref], hyp, smoothing_function=smooth))
# avg_bleu_zero_shot = sum(bleu_zero_shot_scores) / len(bleu_zero_shot_scores) if bleu_zero_shot_scores else 0

# print(f"Average BLEU Score (Few-Shot): {avg_bleu_few_shot:.4f}")
# print(f"Average BLEU Score (Zero-Shot): {avg_bleu_zero_shot:.4f}")

# # --- Calculate ROUGE Scores ---
# rouge_few_shot_scores = {'rouge1': {'fmeasure': 0, 'precision': 0, 'recall': 0},
#                          'rouge2': {'fmeasure': 0, 'precision': 0, 'recall': 0},
#                          'rougeL': {'fmeasure': 0, 'precision': 0, 'recall': 0}}
# count_few_shot = 0

# for item in few_shot_results:
#     reference = item['ground_truth_summary']
#     hypothesis = item['instructblip_summary']
#     if reference and hypothesis: # Ensure both are not empty
#         scores = scorer.score(reference, hypothesis)
#         for key in rouge_few_shot_scores:
#             rouge_few_shot_scores[key]['fmeasure'] += scores[key].fmeasure
#             rouge_few_shot_scores[key]['precision'] += scores[key].precision
#             rouge_few_shot_scores[key]['recall'] += scores[key].recall
#         count_few_shot += 1

# if count_few_shot > 0:
#     for key in rouge_few_shot_scores:
#         rouge_few_shot_scores[key]['fmeasure'] /= count_few_shot
#         rouge_few_shot_scores[key]['precision'] /= count_few_shot
#         rouge_few_shot_scores[key]['recall'] /= count_few_shot

# rouge_zero_shot_scores = {'rouge1': {'fmeasure': 0, 'precision': 0, 'recall': 0},
#                           'rouge2': {'fmeasure': 0, 'precision': 0, 'recall': 0},
#                           'rougeL': {'fmeasure': 0, 'precision': 0, 'recall': 0}}
# count_zero_shot = 0

# for item in zero_shot_results:
#     reference = item['ground_truth_summary']
#     hypothesis = item['instructblip_summary']
#     if reference and hypothesis: # Ensure both are not empty
#         scores = scorer.score(reference, hypothesis)
#         for key in rouge_zero_shot_scores:
#             rouge_zero_shot_scores[key]['fmeasure'] += scores[key].fmeasure
#             rouge_zero_shot_scores[key]['precision'] += scores[key].precision
#             rouge_zero_shot_scores[key]['recall'] += scores[key].recall
#         count_zero_shot += 1

# if count_zero_shot > 0:
#     for key in rouge_zero_shot_scores:
#         rouge_zero_shot_scores[key]['fmeasure'] /= count_zero_shot
#         rouge_zero_shot_scores[key]['precision'] /= count_zero_shot
#         rouge_zero_shot_scores[key]['recall'] /= count_zero_shot

# print("\nAverage ROUGE Scores (Few-Shot):")
# for key, scores in rouge_few_shot_scores.items():
#     print(f"  {key}: F1={scores['fmeasure']:.4f}, P={scores['precision']:.4f}, R={scores['recall']:.4f}")

# print("\nAverage ROUGE Scores (Zero-Shot):")
# for key, scores in rouge_zero_shot_scores.items():
#     print(f"  {key}: F1={scores['fmeasure']:.4f}, P={scores['precision']:.4f}, R={scores['recall']:.4f}")
